In [1]:
import os

emails_dir = r'C:\Users\MGroup\Downloads\Автоматизация-20240128T094908Z-001\Автоматизация'

def get_eml_files(directory):
    files = os.listdir(directory)
    eml_files = [file for file in files if file.endswith('.eml')]
    return eml_files

eml_files = get_eml_files(emails_dir)
eml_files

['Re Offer №453757 13.11.2023, Contrinex _ FAMAGA Group GmbH _ Co. KG .eml',
 'Re ORDER_ Zamowienie Reminder FAMAGA Group GmbH _ Co. KG № KP440822 (5) (2).eml',
 'Re Reminder FAMAGA Group GmbH _ Co. KG № KP434876 (6).eml',
 'Re Reminder FAMAGA Group GmbH _ Co. KG № KP445605 (1).eml']

In [76]:
import email
from email.policy import default

def extract_html_from_eml(file_path):
    # Read the .eml file content
    with open(file_path, 'rb') as file:
        eml_content = file.read()
    
    # Parse the email content
    message = email.message_from_bytes(eml_content, policy=policy.default)
    # message = email.message_from_string(eml_content, policy=default)
    
    # Initialize an empty string to hold the HTML part
    html_content = ""
    
    # Check if the email message is multipart
    if message.is_multipart():
        # Iterate over each part of the email
        for part in message.walk():
            # Check if the content type is HTML
            if part.get_content_type() == 'text/html':
                # Get the HTML content and break the loop
                html_content = part.get_payload(decode=True).decode(part.get_content_charset())
                break
    else:
        # If the email is not multipart, directly check if it's HTML
        if message.get_content_type() == 'text/html':
            html_content = message.get_payload(decode=True).decode(message.get_content_charset())
    
    return html_content

# Example usage (Make sure to replace 'path_to_your_eml_file.eml' with the actual file path)
file_name = 'Re Reminder FAMAGA Group GmbH _ Co. KG № KP434876 (6).eml'
file_path = os.path.join(emails_dir, file_name)
html_content = extract_html_from_eml(file_path)
# print(html_content)


In [ ]:
html_content

In [75]:
from bs4 import BeautifulSoup, NavigableString, Tag

def find_root_blockquotes(soup):
    """
    Find all blockquote elements that are not nested within another blockquote.
    """
    root_blockquotes = []
    for blockquote in soup.find_all("blockquote"):
        if not blockquote.find_parent("blockquote"):
            root_blockquotes.append(blockquote)
    return root_blockquotes

def extract_content_and_nested(blockquote, depth=0):
    """
    Extract content and nested blockquotes, handling various tag arrangements.
    """
    contents = []
    for element in blockquote.contents:
        if isinstance(element, NavigableString):
            if element.strip():  # Ignore empty or whitespace strings
                contents.append((depth, element.strip()))
        elif isinstance(element, Tag):
            if element.name == "blockquote":
                # Recursively process nested blockquotes
                contents.extend(extract_content_and_nested(element, depth + 1))
                break
            else:
                # Process non-blockquote tags for text
                text = element.get_text(strip=True)
                if text:
                    contents.append((depth, text))
                # Also, look for nested blockquotes within other tags like div
                for nested_blockquote in element.find_all("blockquote"):
                    contents.extend(extract_content_and_nested(nested_blockquote, depth + 1))
    return contents

# Parse the HTML content
soup = BeautifulSoup(html_content, "html.parser")

# Find root blockquotes
root_blockquotes = find_root_blockquotes(soup)

# Extract content and nested blockquotes
messages = []
for blockquote in root_blockquotes[:1]:
    messages.extend(extract_content_and_nested(blockquote))

# Print extracted messages
for depth, content in messages:
    print(f"Message at depth {depth}: {content}")


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
messages

In [39]:
blockquotes = soup.find_all("blockquote")
len(blockquotes)

13

In [78]:
def extract_messages_from_body(html_content: str):
    soup = BeautifulSoup(html_content, "html.parser")
    messages = []
    body = soup.find_all("body")[0]
    clone_body = copy.copy(body)
    
    for nested_blockquote in clone_body.find_all("blockquote"):
        nested_blockquote.decompose()
    messages.append(clone_body.get_text(strip=True))
    blockquotes = soup.find_all("blockquote")

    for blockquote in blockquotes:
        clone = copy.copy(blockquote)
        
        for nested_blockquote in clone.find_all("blockquote"):
            nested_blockquote.decompose()
        
        messages.append(clone.get_text(strip=True))

    return messages


In [79]:
messages = extract_messages_from_body(html_content)

In [85]:
for msg in messages:
    if msg and not msg.isspace():
        print(f'Message: {len(msg)}\n\n')

Message: 523


Message: 256


Message: 491


Message: 111


Message: 578


Message: 60


Message: 560


Message: 554


Message: 600


Message: 1680


Message: 4543


Message: 1556




In [ ]:
email_parsed_messages_consistency = [
 {
    "start": "Good",
     "end": "https://famaga.de/"
 },
 {
    "start": "Dear",
    "end": "Amir"
 }
]

11

In [64]:
import hashlib

def normalize_content(content: str) -> str:
    """
    Normalize the content for consistent hashing.
    This function can be expanded based on your normalization needs.
    """
    # Example normalization: strip leading/trailing whitespace and convert to lowercase.
    return content.strip().lower()

def content_hash(content: str) -> str:
    """
    Convert the normalized content to a SHA-256 hash.
    """
    normalized_content = normalize_content(content)
    # Encode the normalized content to a byte format as required by hashlib
    content_bytes = normalized_content.encode()
    # Generate SHA-256 hash
    hash_obj = hashlib.sha256(content_bytes)
    # Convert the hash object to a hexadecimal string
    hash_hex = hash_obj.hexdigest()
    return hash_hex

In [63]:
content_hash(messages[0])

'8218a337c20fd774501ecfb94cb77184d206a6cc324c39b535ba41cefc6043be'

In [68]:
messages

['Very dear Mr. Janura, attached is an updated proforma with the\n      delivery price added. Please send me the payment confirmation\n      after it will be made and we will send your order as fast as\n      possible.Wish you a nice day!-- \nMit freundlichen Grüßen / Best regards / Z wyrazami szacunku,\nSawwa Wronskiy\nVertrieb / Sales Manager\nFAMAGA Group OHG\n\n+48226022408https://famaga.de/',
 'Hello\n\nWe decide to deliver our order using your transport.\nPlease send me a pro-forma invoicePozdrawiam\n              / Best regardsAdam\n          JanuraKom.\n                  / Mobile+48 537-797-300www.landoia.pl"LANDOIA"\n              Kapusta Łukasz26-613\n              Radom,ul.\n              Marii Gajl 1NIP\n              7962440697W dniu 2023-11-22 08:57, Sawwa Vronskiy\n        napisał(a):',
 'Good day,Very dear Mr. Janura, sure, we have calculated shipping\n              and it made by us is will be 159.24 euro. If you would\n              like us to made it I will prepare y

In [ ]:
import re


for m in messages:
    print(re.sub(r'\s+', ' ', m).strip())
    print('\n\n')

In [44]:
body = soup.find_all("body")[0]

for nested_blockquote in body.find_all("blockquote"):
    nested_blockquote.decompose()

print(body.get_text(strip=True))
print('\n\n-----------------------------------------------------------------\n\n')

Very dear Mr. Janura, attached is an updated proforma with the
      delivery price added. Please send me the payment confirmation
      after it will be made and we will send your order as fast as
      possible.Wish you a nice day!-- 
Mit freundlichen Grüßen / Best regards / Z wyrazami szacunku,
Sawwa Wronskiy
Vertrieb / Sales Manager
FAMAGA Group OHG

+48226022408https://famaga.de/


-----------------------------------------------------------------




In [ ]:
for blockquote in blockquotes:
    # Clone the blockquote to avoid modifying the original
    clone = copy.copy(blockquote)
    
    # Remove any nested blockquote elements from the clone
    for nested_blockquote in clone.find_all("blockquote"):
        nested_blockquote.decompose()
    
    # Extract and print the content of the blockquote without the nested ones
    print(clone.get_text(strip=True))
    print('\n\n-----------------------------------------------------------------\n\n')

In [ ]:
html_content

In [25]:
soup = BeautifulSoup(html_content, "html.parser")

In [28]:
root_blockquotes = []
for blockquote in soup.find_all("blockquote"):
    if not blockquote.find_parent("blockquote"):
        root_blockquotes.append(blockquote)

len(root_blockquotes)

root_blockquote = root_blockquotes[0]

In [32]:
root_blockquote.contents[0]

isinstance(root_blockquote.contents[0], NavigableString)

True

In [ ]:
 elif isinstance(element, Tag):
            if element.name == "blockquote":
                # Recursively process nested blockquotes
                contents.extend(extract_content_and_nested(element, depth + 1))
            else:
                # Process non-blockquote tags for text
                text = element.get_text(strip=True)
                if text:
                    contents.append((depth, text))
                # Also, look for nested blockquotes within other tags like div
                for nested_blockquote in element.find_all("blockquote"):
                    contents.extend(extract_content_and_nested(nested_blockquote, depth + 1))

In [16]:
from bs4 import BeautifulSoup, NavigableString, Tag
import copy


def extract_content_and_nested_blockquotes(blockquote, depth=0):
    """
    Extracts content from a blockquote, saves and removes nested blockquotes,
    and then processes them recursively.
    """
    # Step 2: Save nested blockquotes and remove them from the parent
    nested_blockquotes = blockquote.find_all("blockquote", recursive=False)
    for nested in nested_blockquotes:
        nested.extract()  # This removes the nested blockquote from the parent

    # Step 3: Get content from the parent blockquote
    # Here, we're considering any text outside of the nested blockquotes as the parent's content
    content = blockquote.get_text(strip=True)
    messages = [(depth, content)] if content else []

    # Step 4: Recursively do the same for each child blockquote
    for nested in nested_blockquotes:
        messages += extract_content_and_nested_blockquotes(nested, depth+1)

    return messages


soup = BeautifulSoup(html_content, "html.parser")

# Step 1: Get blockquote (starting with the root blockquotes)
root_blockquotes = soup.find_all("blockquote", recursive=False)

# Extract messages and their nesting levels from each root blockquote
messages = []
for blockquote in root_blockquotes:
    messages += extract_content_and_nested_blockquotes(blockquote)

# Print extracted messages
for depth, message in messages:
    print(f"Depth: {depth}, Message: {message}")

In [20]:
root_blockquotes


[]

In [ ]:
html_content

In [ ]:
from bs4 import BeautifulSoup


def strip_tags_correctly(soup):
    """
    Recursively remove text from a BeautifulSoup tag structure
    while keeping the tags themselves.
    """
    if soup.string:
        soup.string.replace_with("")
    for tag in soup.find_all(True):
        strip_tags_correctly(tag)
    return soup


# Parse the HTML
soup = BeautifulSoup(html_content, "html.parser")

# Strip content while keeping the structure
stripped_soup = strip_tags_correctly(soup)

print(stripped_soup.prettify())

In [ ]:
body
